In [ ]:
import random
import numpy as np
import time
from funky_lights import messages
from functools import reduce
from operator import add

In [ ]:
TEXTURE_WIDTH = 128
TEXTURE_HEIGHT = 128
TEXTURE_SIZE = TEXTURE_WIDTH * TEXTURE_HEIGHT * 4

class Segment:
    def __init__(self, uid, num_leds, led_positions):
        self.uid = uid
        self.num_leds = num_leds
        self.colors = np.array([[255, 0, 0] for i in range(num_leds)], dtype=np.ubyte)
        self.led_positions = np.array(led_positions)

def PrepareTextureMsg(segments):
    msg = []
    texture_size = TEXTURE_SIZE
    for segment in segments:
        for color in segment.colors:
            msg += [color[0], color[1], color[2], 255]
    msg += [0] * (texture_size - len(msg))
    return bytearray(msg)


def PrepareTextureMsg_Opt(segments):
    all_colors = []
    for segment in segments:
        alpha = np.zeros((segment.colors.shape[0], 1), dtype=np.uint8)
        colors = np.append(segment.colors, alpha, axis=1)
        all_colors.append(colors)   
    bytes = b"".join([colors.tobytes() for colors in all_colors])
    bytes += bytearray(TEXTURE_SIZE - len(bytes))
    return bytearray(bytes)


def PrepareTextureMsg_Opt2(segments):
    out = np.zeros(TEXTURE_SIZE, dtype=np.uint8)
    r = np.concatenate([segment.colors[:, 0] for segment in segments], axis=None)
    g = np.concatenate([segment.colors[:, 1] for segment in segments], axis=None)
    b = np.concatenate([segment.colors[:, 2] for segment in segments], axis=None)
    color_bytes = reduce(add, [segment.num_leds for segment in segments]) * 4
    out[:color_bytes:4] = r
    out[1:color_bytes:4] = g
    out[2:color_bytes:4] = b
    return bytearray(out)

def PrepareTextureMsg_Opt3(segments):
    bytes = b"".join([segment.colors.tobytes() for segment in segments])
    bytes += bytearray(TEXTURE_SIZE - len(bytes))
    return bytearray(bytes)


segments = []
for i in range(50):
    num_leds = 100
    led_positions = [[[random.randrange(1), random.randrange(1), random.randrange(1)]] for x in range(num_leds)]
    segment = Segment(i, num_leds, led_positions)
    segments.append(segment)

start = time.time()
for i in range(1000):
    out = PrepareTextureMsg(segments)
stop = time.time()
print("Old method: {} secs".format(stop - start))


start = time.time()
for i in range(1000):
    out = PrepareTextureMsg_Opt2(segments)
stop = time.time()
print("New method: {} secs".format(stop - start))